In [156]:
import enum

import isort
import sys
import platform
import importlib
from types import ModuleType
import inspect

In [157]:
def get_stdlib_packages():
    if sys.version_info.minor == 10:
        module_names = sys.stdlib_module_names
    else:
        module_names = isort.stdlibs.py38.stdlib

    external_packages = list()
    for name in module_names:
        if name[0] == '_' or name == 'this' or name == 'antigravity':
            continue
        external_packages.append(name)
    return external_packages

def get_real_packages(package_names):
    real_modules = list()
    not_importable_modules = list()
    for name in package_names:
        try:
            importlib.import_module(name)
            real_modules.append(name)
        except:
            not_importable_modules.append(name)
    return real_modules, not_importable_modules


def get_real():
    external_packages = get_stdlib_packages()
    return get_real_packages(external_packages)

In [158]:
def module_dependency(module_names, name):
    if name not in module_names:
        raise Exception(f'{name} is not importable module')
    dp_names = list()

    # try:
    #     importlib.import_module(name)
    # except:
    #     print(f'err name: {name}')
    for key, val in vars(sys.modules[name]).items():
        if isinstance(val, ModuleType):
            md_name = val.__name__

            try:
                index = md_name.index(".")
                md_name = md_name[0:index]
            except:
                pass

            dp_names.append(md_name)
            # print(f'key: {key}, type: {type(val)}, val: {md_name}, module: {md_name}')

    return dp_names

In [159]:
real_modules, _ = get_real()
# print(real_modules)

In [160]:
import pathlib

dp_names = module_dependency(real_modules, 'pathlib')
print(dp_names)

['fnmatch', 'functools', 'io', 'ntpath', 'os', 'posixpath', 're', 'sys']


In [161]:
# import sys
# sys.setrecursionlimit(50000)
#
# def dependency_graph(modules, ad_mtrx, original_modules, count):
#     count = count + 1
#     if count % 10 == 0:
#         print(f'count: {count}')
#     for md in modules:
#         try:
#             dp_names = module_dependency(original_modules, md)
#         except:
#             print(f'{md} is not in stdlib')
#             continue
#
#         for name in dp_names:
#             try:
#                 i = original_modules.index(md)
#                 j = original_modules.index(name)
#                 ad_mtrx[i][j] = 1
#             except:
#                 print(f'md: {md}, name: {name} is not in stdlib')
#                 continue
#
#         dependency_graph(dp_names, ad_mtrx, original_modules, count)
#
# n = len(real_modules)
# ad_mtrx = [[0 for x in range(n)] for y in range(n)]
# dependency_graph(real_modules, ad_mtrx, real_modules, 0)
# print(ad_mtrx)

In [162]:
dp = module_dependency(real_modules, 'os')
print(dp)

['abc', 'sys', 'stat', 'posixpath']


In [163]:
def module_dependency_map(modules):
    md_map = dict()
    for md in modules:
        md_map[md] = module_dependency(modules, md)
    return md_map

md_map = module_dependency_map(real_modules)
print(md_map)

{'asyncio': ['sys', 'asyncio', 'asyncio', 'asyncio', 'asyncio', 'asyncio', 'asyncio', 'asyncio', 'asyncio', 'asyncio', 'asyncio', 'asyncio', 'asyncio', 'asyncio', 'asyncio', 'asyncio', 'asyncio', 'asyncio', 'asyncio', 'asyncio', 'asyncio', 'asyncio', 'asyncio', 'asyncio', 'asyncio'], 'ipaddress': ['functools'], 'zipimport': ['importlib', 'importlib', '_imp', 'io', 'marshal', 'sys', 'time'], 'copy': [], 'functools': [], 'importlib': ['_imp', 'sys', 'importlib', 'importlib', 'types', 'warnings', 'importlib', 'importlib', 'importlib'], 'mailbox': ['os', 'time', 'calendar', 'socket', 'errno', 'copy', 'warnings', 'email', 'io', 'contextlib', 'fcntl'], 'nis': [], 'asynchat': ['asyncore'], 'base64': ['re', 'struct', 'binascii'], 'io': ['io', 'abc'], 'binascii': [], 'xmlrpc': ['xmlrpc', 'xmlrpc'], 'tempfile': ['functools', 'warnings', 'io', 'os', 'shutil', 'errno', 'sys', 'weakref', '_thread'], 'socketserver': ['socket', 'selectors', 'os', 'sys', 'threading'], 'subprocess': ['builtins', 'errno

In [164]:
def build_adj_edge_graph(md_map):
    index_list = list()

    for k, v in md_map.items():
        index_list.append(k)

    n = len(index_list)
    adj_list = list()
    for i in range(n):
        adj_list.append([])

    for md in index_list:
        for name in md_map[md]:
            try:
                i = index_list.index(md)
                j = index_list.index(name)
                if j not in adj_list[i]:
                    adj_list[i].append(j)
            except:
                continue

    return adj_list, index_list

In [165]:
adj_list, index_list = build_adj_edge_graph(md_map)
print(adj_list)
# print(index_list)

[[157, 0], [4], [5, 10, 161, 157, 64], [], [], [157, 5, 48, 145], [165, 64, 188, 66, 40, 3, 145, 89, 10, 59, 58], [], [159], [25, 95, 11], [10, 199], [], [12], [4, 145, 10, 165, 128, 40, 157, 127], [66, 80, 165, 157, 139], [114, 40, 10, 165, 64, 100, 157, 139, 145, 59, 195, 80], [16], [29, 10, 56, 25, 157], [114, 10, 165], [], [10, 165, 95, 11], [11, 165, 157], [9, 11, 165], [], [157, 165, 158, 149, 177, 13], [33, 126, 144, 4, 93], [], [10, 165, 145], [116, 132, 165, 56], [114, 157], [175, 114, 19], [157, 66, 80], [], [157], [165, 25, 116, 157], [75, 166, 132, 157], [], [10, 165, 95, 157, 37], [165], [], [], [157, 165, 181], [116, 4, 10, 190, 165, 53, 25, 157], [], [75, 165, 25, 157], [120, 157], [157, 48, 75, 10], [11, 29, 59, 69, 33, 166, 165, 25, 95], [], [29, 157, 49], [184, 157, 64, 166], [5, 165, 157, 146, 48, 145], [157, 165, 114, 10], [165, 157, 171], [145, 23], [157, 66, 163], [157, 25, 10, 29], [177, 165, 25, 157], [], [199, 157, 75], [157, 120], [], [114, 5, 84, 10, 165, 76,

In [166]:
# def build_dependency_graph(md_map):
#     index_list = list()
#
#     for k, v in md_map.items():
#         index_list.append(k)
#
#     n = len(index_list)
#     ad_mtrx = [[0 for x in range(n)] for y in range(n)]
#
#     for md in index_list:
#         for name in md_map[md]:
#             try:
#                 i = index_list.index(md)
#                 j = index_list.index(name)
#                 ad_mtrx[i][j] = 1
#             except:
#                 continue
#
#     return index_list, ad_mtrx
#
#
# index_list, ad_mtrx = build_dependency_graph(md_map)
# # print(f'index list: {index_list}')
# # print(f'ad_mtrx: {ad_mtrx}')
#
# i = index_list.index('locale')
# print(f'dp of locale ---------')
# dp_list = list()
# for j in range(len(index_list)):
#     if ad_mtrx[i][j] == 1:
#         dp_list.append(index_list[j])
# print(dp_list)

In [167]:
dp = module_dependency(real_modules, 'lzma')
print(dp)

# print(index_list.index('builtins')

['builtins', 'io', 'os', '_compression']


In [168]:
def print_cycle(stack, v):
    print(f'stack: {stack}')
    st2 = [stack.pop()]
    while st2[-1] != v:
        st2.append(stack.pop())

    rs = [v]
    while len(st2) > 0:
        # print(st2[-1])
        rs.append(st2[-1])
        stack.append(st2[-1])
        st2.pop()

    print(f'cycle: {rs}')


In [169]:
def process_DFS_tree(graph, stack, visited_vertices):
    for v in graph[stack[-1]]:
        if visited_vertices[v] == 'in_stack':
            print_cycle(stack, v)
        elif visited_vertices[v] == 'not_visited':
            stack.append(v)
            visited_vertices[v] = 'in_stack'
            process_DFS_tree(graph, stack, visited_vertices)

    visited_vertices[stack[-1]] = 'done'
    stack.pop()

In [170]:
def find_cycles(graph):
    n = len(graph)
    visited = list()
    for i in range(n):
        visited.append('not_visited')

    for v in range(n):
        if visited[v] == 'not_visited':
            stack = [v]
            visited[v] = 'in_stack'
            process_DFS_tree(graph, stack, visited)


In [171]:
adj_list, index_list = build_adj_edge_graph(md_map)
find_cycles(adj_list)

stack: [0]
cycle: [0, 0]
stack: [2, 5]
cycle: [5, 5]
stack: [2, 10]
cycle: [10, 10]
stack: [6, 165, 53]
cycle: [165, 165, 53]
stack: [6, 188, 177, 49]
cycle: [49, 49]
stack: [6, 188, 177, 75]
cycle: [75, 75]
stack: [6, 89]
cycle: [89, 89]
stack: [12]
cycle: [12, 12]
stack: [16]
cycle: [16, 16]
stack: [24, 158]
cycle: [158, 158]
stack: [24, 149]
cycle: [149, 149]
stack: [37]
cycle: [37, 37]
stack: [68]
cycle: [68, 68]
stack: [71]
cycle: [71, 71]
stack: [85]
cycle: [85, 85]
stack: [87]
cycle: [87, 87]
stack: [97]
cycle: [97, 97]
stack: [108]
cycle: [108, 108]
stack: [113]
cycle: [113, 113]
stack: [150]
cycle: [150, 150]


In [172]:
# print(index_list[2], index_list[5])

print(index_list[6])
print(index_list[165])
print(index_list[53])
print(index_list[12])
# print(index_list[6])

zipimport importlib
mailbox
os
posixpath
xmlrpc


In [173]:
import mailbox
import os
import posixpath
import xmlrpc

import zipimport
import importlib